In [161]:
import numpy as np
import pandas as pd
from data_reader import read_consumption_and_weather
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
import math

torch.manual_seed(1)

In [162]:
_, _, df = read_consumption_and_weather()

df1 = df['NO1']
df2 = df['NO2']
df3 = df['NO3']
df4 = df['NO4']
df5 = df['NO5']

Holes by length and occurrences in column NO3:
holes
1        1
dtype: int64
Filling holes up to length 3
Any remaining holes after interpolation? False


In [163]:
df = df1[['consumption']]

df

,consumption
2017-05-01 00:00:00+00:00,3325.431995
2017-05-01 01:00:00+00:00,3344.690998
2017-05-01 02:00:00+00:00,3398.359002
2017-05-01 03:00:00+00:00,3430.220001
2017-05-01 04:00:00+00:00,3606.750000
...,...
2023-01-21 19:00:00+00:00,5947.502808
2023-01-21 20:00:00+00:00,5868.196111
2023-01-21 21:00:00+00:00,5742.156776
2023-01-21 22:00:00+00:00,5653.673398


In [164]:
# Get the 0.8 first timesteps for training and 0.2 last one for testing
df_train, df_test = train_test_split(df, test_size=0.1, shuffle=False)
# Divide into train, validation
df_train, df_validation = train_test_split(df_train, test_size=0.1, shuffle=False)

In [165]:
# Save for undoing normalization in testing
train_mean = df_train.mean()
train_std = df_train.std()

# Normalize train and test set
df_train = (df_train - train_mean) / train_std
df_validation = (df_validation - train_mean) / train_std
df_test = (df_test - train_mean) / train_std

In [166]:
df_train

,consumption
2017-05-01 00:00:00+00:00,-0.520736
2017-05-01 01:00:00+00:00,-0.506271
2017-05-01 02:00:00+00:00,-0.465960
2017-05-01 03:00:00+00:00,-0.442029
2017-05-01 04:00:00+00:00,-0.309436
...,...
2021-12-20 07:00:00+00:00,1.151071
2021-12-20 08:00:00+00:00,1.168536
2021-12-20 09:00:00+00:00,1.153706
2021-12-20 10:00:00+00:00,1.116430


In [167]:
def get_src_trg(sequence, enc_seq_len, pw):
    src = sequence[:enc_seq_len]
    trg = sequence[enc_seq_len-1:enc_seq_len+pw-1]
    trg_y = sequence[-pw:]

    return src, trg, trg_y


def generate_sequences(df, enc_seq_len, pw):    
    data = dict()
    L = len(df)
    for i in range(L-enc_seq_len):
        sequence = df[i:i+enc_seq_len+pw].values 

        src, trg, trg_y = get_src_trg(sequence, enc_seq_len, pw)

        data[i] = {'src': src, 'trg': trg, 'trg_y': trg_y}

    return data

In [168]:
class SequenceDataset(torch.utils.data.Dataset):

    def __init__(self, df):
        self.data = df

    def __getitem__(self, idx):
        sample = self.data[idx]
        return torch.Tensor(sample['src']), torch.Tensor(sample['trg']), torch.Tensor(sample['trg_y'])
    
    def __len__(self):
        return len(self.data)

In [169]:
tw = 100
pw = 1

train_dataset = SequenceDataset(generate_sequences(df_train, tw, pw))
val_dataset = SequenceDataset(generate_sequences(df_validation, tw, pw))
test_dataset = SequenceDataset(generate_sequences(df_test, tw, pw))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True, drop_last=True)

In [170]:
for src, trg, trg_y in train_loader:
    print(trg_y.shape)

torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size([32, 1, 1])
torch.Size(

In [171]:
# For use in the transformer model
class PositionalEncoder(nn.Module):

    def __init__(self, dropout=0.1, max_seq_len=5000, d_model=512, batch_first=True):
        super().__init__()

        self.d_model = d_model
        self.dropout = nn.Dropout(p=dropout)
        self.batch_first = batch_first
        self.x_dim = 1 if batch_first else 0

        position = torch.arange(max_seq_len).unsqueeze(1)

        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))

        pe = torch.zeros(max_seq_len, 1, d_model)

        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe)

    
    def forward(self, x):
        x = x + self.pe[:x.size(self.x_dim)]
        return self.dropout(x)
    

In [172]:

class TransformerModel(nn.Module):

    def __init__(self):
        super().__init__()

        enc_input_size = 100
        n_features = 1
        dec_input_size = 1
        output_size = 1
        dim=512

        # ENCODER
        self.encoder_input_layer = nn.Linear(
            in_features=n_features,
            out_features=dim
        )

        self.positional_encoding_layer = PositionalEncoder(
            d_model=dim,
            dropout=0.1,
            max_seq_len=5000
        )

        n_heads = 8

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=dim,
            nhead=n_heads,
            dim_feedforward=2028,
            dropout=0.2,
            batch_first=True
        )

        self.encoder = nn.TransformerEncoder(
            encoder_layer=encoder_layer,
            num_layers=4,
            norm=None
        )

        # DECODER
        self.decoder_input_layer = nn.Linear(
            in_features=dec_input_size,
            out_features=dim
        )

        decoder_layer = nn.TransformerDecoderLayer(
            d_model=dim,
            nhead=n_heads,
            dim_feedforward=2048,
            dropout=0.2,
            batch_first=True
        )

        self.decoder = nn.TransformerDecoder(
            decoder_layer=decoder_layer,
            num_layers=4,
            norm=None
        )

        self.linear_mapping = nn.Linear(
            in_features=dim,
            out_features=output_size
        )


    def forward(self, src, tgt, src_mask, tgt_mask):

        print("Input src size: ", src.shape)

        src = self.encoder_input_layer(src)

        src = self.positional_encoding_layer(src)

        src = self.encoder(src=src)

        decoder_output = self.decoder_input_layer(tgt)

        decoder_output = self.decoder(
            tgt=decoder_output,
            memory=src,
            tgt_mask=tgt_mask,
            memory_mask=src_mask
        )

        decoder_output = self.linear_mapping(decoder_output)

        return decoder_output



In [173]:
# Generate masking for decoder input
def generate_mask(dim1, dim2):
    mask = torch.triu(torch.ones(dim1, dim2) * float('-inf'), diagonal=1)
    return mask

In [174]:
model = TransformerModel()

In [175]:
# Training Loop

optimizer = torch.optim.Adam(model.parameters())

criterion = torch.nn.MSELoss()

for epoch in range(10):

    for i, (src, trg, trg_y) in enumerate(train_loader):

        optimizer.zero_grad()

        # Create masks
        src_mask = generate_mask(pw, tw)

        trg_mask = generate_mask(pw, pw)

        # Forecast
        prediction = model(src, trg, src_mask, trg_mask)

        # Loss and backprop
        loss = criterion(trg_y, prediction)

        loss.backward()

        optimizer.step()

    # Validation
    model.eval()

    with torch.no_grad():
        for i, (src, _, trg_y) in enumerate(val_loader):

            prediction = 0

Input src size:  torch.Size([32, 100, 1])


RuntimeError: The size of tensor a (32) must match the size of tensor b (100) at non-singleton dimension 0